# CA RNN
Toy example in 1d


In [3]:
'''
Example
'''
import tensorflow as tf
import numpy as np

import utils


class GRUCell(object):
    '''Standard GRU cell.
    
    Calling this cell on a RNN network returns a tuple of activations (h, h)
    '''
    def __init__(self, state_size, activation=tf.sigmoid):
        self._state_size = state_size
        self._activation = activation

    @property
    def state_size(self):
        return self._state_size
    
#     def zero_state(self, batch_size):
#         return 
        
    def __call__(self, s, scope=None):
        with tf.name_scope('GRUCell'):
            
            # trainable variables
            with tf.variable_scope('weights', initializer=tf.contrib.layers.xavier_initializer()) as scope:
                W_u = tf.get_variable('W_u', [self._state_size, self._state_size])
                W_r = tf.get_variable('W_r', [self._state_size, self._state_size])
                W   = tf.get_variable('W', [self._state_size, self._state_size])
                b   = tf.get_variable('b', [self._state_size], initializer=tf.constant_initializer(1.0))
            scope.reuse_variables()
            
            # u = update, r = reset, c = candidates, h = new hidden states
            u = self._activation(tf.matmul(s, W_u) + b)
            r = self._activation(tf.matmul(s, W_r) + b)
            c = tf.tanh(tf.matmul(tf.multiply(s, r), W) + b)
            h = tf.multiply(s, u) + tf.multiply((1 - u), c)
            return h, h

        
class ca_rnn(object):
    '''
    A CA RNN neural network tensorflow model.
    
    Aims to generalise a CA algorithm from training data
    
    Properties:
        inference:
        loss:
        optimizer:
        prediction:
    '''
    def __init__(self,
                 learning_rate=1e-4,
                 state_size=5,
                 batch_size=1):
        self._learning_rate = learning_rate
        self._state_size = state_size
        self._batch_size = batch_size
        
        # Inference
        with tf.name_scope('inference'):
            width = 5
            depth = 1
            classes = 2
            time_steps = 1
            
            with tf.name_scope('input'):
                x, y = self.input_pipeline('constrained_100000x5x1x1.tfrecord', batch_size, 2)
                self._x = x
                self._y = y

            # RNN
            with tf.name_scope('rnn'):
                # GRU cell
                cell = GRUCell(state_size)

                # Just one time step
                zero_state = tf.reshape(x, [batch_size, width])  # load all X on state zero
#                 zero_state = x
                output, state = cell(zero_state)

            # Convolution
            with tf.name_scope('convolution'):
                output = tf.reshape(output, [batch_size, width, depth])
                kernel = tf.Variable(tf.random_normal([3, 1, 1]), dtype=tf.float32, name='kernel')
                conv = tf.nn.conv1d(output, kernel, stride=1, padding='SAME')

            # FC1
            with tf.name_scope('fc1'):
                with tf.variable_scope('output'):
                    W = tf.get_variable('W_out', [state_size, classes])
                    b = tf.get_variable('b_out', [classes], initializer=tf.constant_initializer(1.0))

                flatten = tf.reshape(conv, [batch_size, width])
                fc1 = tf.nn.xw_plus_b(flatten, W, b, name='fc1')
                fc1 = tf.reshape(fc1, [batch_size, classes])

            self._logits = fc1
        
        # loss function
        with tf.name_scope('loss'):
            cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=self._logits, labels=y, name='cross_entropy')
            self._loss = tf.reduce_mean(cross_entropy)
            # add scalar histogram for loss
            
            self._optimizer = tf.train.AdamOptimizer(self._learning_rate).minimize(self._loss)
        
        # evaluation
        with tf.name_scope('prediction'):
            correct = tf.nn.in_top_k(self._logits, y, 1)
            self._prediction = tf.reduce_mean(tf.cast(correct, tf.float32))
        
    @property
    def inference(self):
        return self._logits
    
    @property
    def optimizer(self):
        return self._optimizer
    
    @property
    def loss(self):
        return self._loss
    
    @property
    def prediction(self):
        return self._prediction
    

# Reset tensorflow variable names
tf.reset_default_graph()


# Hyperparameters
epochs = 1


# input pipeline
filename_queue = tf.train.string_input_producer(
    ['constrained_100000x5x1x1.tfrecords'], num_epochs=epochs)


# read and decode   
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)

data = tf.parse_single_example(
    serialized_example, 
    features={
        'x': tf.FixedLenFeature([], tf.string),
        'y': tf.FixedLenFeature([], tf.int64)
    })

# decode
width = 5
height = 1
depth = 1

features = tf.decode_raw(data['x'], tf.int64)
features = tf.reshape(features, [width, height, depth])
features = tf.cast(features, dtype=tf.float32)
labels = tf.cast(data['y'], dtype=tf.int32)


batch_size = 256
min_after_dequeue = 1000
capacity = min_after_dequeue + 3 + batch_size
example_batch, label_batch = tf.train.shuffle_batch(
    [features, labels], 
    batch_size=batch_size, 
    capacity=capacity,
    allow_smaller_final_batch=True,
    min_after_dequeue=min_after_dequeue)


# construct model
# learning_rate = 0.01
# model = ca_rnn(batch_size=batch_size, learning_rate=learning_rate)


# train network
init = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()  # for epochs local variable

num_examples = 0
with tf.Session() as sess:
    sess.run([init, init_local])

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#     writer = tf.summary.FileWriter('./graphs/run1', sess.graph)

    try:
        step = 0
        while not coord.should_stop():
            xx, yy = sess.run([example_batch, label_batch])
            print(xx.shape, yy.shape)

            num_examples = num_examples + xx.shape[0]
            print('num_examples: ', num_examples)

    except tf.errors.OutOfRangeError:
#         print('Done training for %d epochs, %d steps.' % (num_epochs, step))
        print('Requesting to stop')
        coord.request_stop()
    finally:
        # ask threads to to stop
        coord.request_stop()      

        # wait for threads to finish
        coord.join(threads)

#     writer.close()


(256, 5, 1, 1) (256,)
num_examples:  256
(256, 5, 1, 1) (256,)
num_examples:  512
(256, 5, 1, 1) (256,)
num_examples:  768
(256, 5, 1, 1) (256,)
num_examples:  1024
(256, 5, 1, 1) (256,)
num_examples:  1280
(256, 5, 1, 1) (256,)
num_examples:  1536
(256, 5, 1, 1) (256,)
num_examples:  1792
(256, 5, 1, 1) (256,)
num_examples:  2048
(256, 5, 1, 1) (256,)
num_examples:  2304
(256, 5, 1, 1) (256,)
num_examples:  2560
(256, 5, 1, 1) (256,)
num_examples:  2816
(256, 5, 1, 1) (256,)
num_examples:  3072
(256, 5, 1, 1) (256,)
num_examples:  3328
(256, 5, 1, 1) (256,)
num_examples:  3584
(256, 5, 1, 1) (256,)
num_examples:  3840
(256, 5, 1, 1) (256,)
num_examples:  4096
(256, 5, 1, 1) (256,)
num_examples:  4352
(256, 5, 1, 1) (256,)
num_examples:  4608
(256, 5, 1, 1) (256,)
num_examples:  4864
(256, 5, 1, 1) (256,)
num_examples:  5120
(256, 5, 1, 1) (256,)
num_examples:  5376
(256, 5, 1, 1) (256,)
num_examples:  5632
(256, 5, 1, 1) (256,)
num_examples:  5888
(256, 5, 1, 1) (256,)
num_examples:  

In [1]:
import numpy as np
import tensorflow as tf
import utils







xx, yy = input_pipeline('test.tfrecords', batch_size=2, num_epochs=None)
# xx, yy = read_and_decode('test.tfrecords')

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# Start input enqueue threads
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

# first example
x1, y1 = sess.run([xx, yy])
x2, y2 = sess.run([xx, yy])


print(x1.shape, x2.shape)
print(y1, y2)

# try:
#     while not coord.should_stop():
#         # Run training steps
# #         for i in range(5):
#         xxx = sess.run(xx)
#         print(xxx)
# except tf.errors.OutOfRangeError:
#     print('Done training -- epoch limit reached')
# finally:
#     # stop threads
#     coord.request_stop()
    
# finish all threads
coord.join(threads)

sess.close()



Writing test.tfrecords


TypeError: only integer scalar arrays can be converted to a scalar index

In [3]:
import numpy as np
x = np.random.choice(5, size=[1, 5, 1])
print('in: ', x.shape)
x = x.tobytes()
print('x bytes:', x)
x = np.fromstring(x, dtype=np.uint64)
print('before reshape: ', x.shape)
x = np.reshape(x, [1, 5, 1])
print('out:', x.shape)



in:  (1, 5, 1)
x bytes: b'\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00'
before reshape:  (5,)
out: (1, 5, 1)


In [1]:
from utils import generate_constrained_dataset

generate_constrained_dataset('constrained2', [5, 1, 1], 5000, 5000, verbose=True)

Positive boards generated: 100
Positive boards generated: 200
Positive boards generated: 300
Positive boards generated: 400
Positive boards generated: 500
Positive boards generated: 600
Positive boards generated: 700
Positive boards generated: 800
Positive boards generated: 900
Positive boards generated: 1000
Positive boards generated: 1100
Positive boards generated: 1200
Positive boards generated: 1300
Positive boards generated: 1400
Positive boards generated: 1500
Positive boards generated: 1600
Positive boards generated: 1700
Positive boards generated: 1800
Positive boards generated: 1900
Positive boards generated: 2000
Positive boards generated: 2100
Positive boards generated: 2200
Positive boards generated: 2300
Positive boards generated: 2400
Positive boards generated: 2500
Positive boards generated: 2600
Positive boards generated: 2700
Positive boards generated: 2800
Positive boards generated: 2900
Positive boards generated: 3000
Positive boards generated: 3100
Positive boards g